In [1]:
import os
import glob
import pandas as pd

In [3]:
matrix_dir = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/all_samples/results/MatrixEQTL_result"
snp = "%s/final_SNP_reorder.csv" % matrix_dir

In [30]:
df = pd.read_csv(snp, index_col=0)
df.head()

,SRR7075101,SRR1811018,SRR1811012,SRR1182601,SRR1182599,SRR1182597,SRR1182595,SRR1182587-88,SRR1182582-84,SRR5978883-89,...,SRR494616,SRR494614,SRR5934169,SRR456936,SRR7130854,SRR7130859,SRR7130862,SRR7130864,SRR7130867,SRR7130874
id,,,,,,,,,,,,,,,,,,,,,
chr1:15774,0,1,0,1,1,1,1,1,1,0,...,0,0,0,1,0,0,1,1,0,0
chr1:64649,0,0,1,1,1,1,1,0,1,0,...,0,0,0,0,1,0,1,1,0,1
chr1:67631,0,0,1,1,0,1,1,1,1,0,...,0,0,0,0,1,0,0,0,0,1
chr1:70317,0,0,1,1,0,1,0,1,1,0,...,0,0,0,0,0,0,1,0,0,1
chr1:78942,0,1,0,0,0,1,0,1,1,0,...,0,0,0,1,1,0,1,1,0,1


#### the proportion of QTL SNPs identified from heterogyzous and homogyzous

In [17]:
cis_file = "%s/m6AQTL_cis_linear_v8.txt" % matrix_dir
df_cis = pd.read_table(cis_file)
df_cis.head()
cis_list = df_cis["SNP"].tolist()

In [32]:
#df_het.head()
df_cis_geno = df[df.index.isin(cis_list)]
print(len(df_cis_geno))
df_cis_geno.head()

df_cis_geno_het = df_cis_geno[df_cis_geno.isin({1}).any(1)]
df_cis_geno_hom = df_cis_geno[~(df_cis_geno.isin({1}).any(1))]
print(len(df_cis_geno_het), len(df_cis_geno_hom))

4847
3626 1221


#### the proportion of heterogyzous QTL SNPs didn't appear in ASE analysis
##### only those SNPs which located within the ip samples (with reads cover) were considered.

In [36]:
df_cis_geno_het.head()
cis_het_list = df_cis_geno_het.index
print(cis_het_list[0])

chr1:890295


In [43]:
ip_dir = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/all_samples/results/ASE_analysis/ASE_readCount/ip_count"
ip_total_file = "%s/ip_total.count" % ip_dir
#
input_dir = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/all_samples/results/ASE_analysis/ASE_readCount/input_count"
input_total_file = "%s/input_total.count" % input_dir
###########################################

def get_snps(in_file):
    df = pd.read_table(in_file)
    df['SNP'] = df.iloc[:, 0] + ":" + df.iloc[:, 1].astype(str)
    total_list = df["SNP"].tolist()
    print(total_list[0])
    return total_list
ip_total_list = get_snps(ip_total_file)
input_total_list = get_snps(input_total_file)

chr18:812170


/home/galaxy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (1,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


chr1:672132


In [45]:
#
print(len(list(set(cis_het_list))), len(list(set(ip_total_list))))
print(len(set(ip_total_list).intersection(set(cis_het_list))))
#
print(len(list(set(cis_het_list))), len(list(set(input_total_list))))
print(len(set(input_total_list).intersection(set(cis_het_list))))
#
print(len(set(input_total_list).intersection(set(ip_total_list))))

3626 753746
160
3626 2701540
424
753358


#### distinguish SNP from input sample and reference imputation (abandon)

In [46]:
diff_list = list(set(cis_het_list).difference(set(ip_total_list)))
print(len(diff_list), diff_list[0])

3466 chr6:36362328


In [ ]:
## before imputation
vcf_dir = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/all_samples/results/input_snp/"
example = "%s/SRR7130896/SRR7130896-filtered_variants_hg19_uniq.vcf"
def get_snp_from_vcf(vcf):
    df = pd.read_table(example, comment="#", header=None)
    df["SNP"] = df.iloc[:, 0] + ":" + df.iloc[:, 1].astype(str)
    snp_list = df["SNP"].tolist()